Before running, install required packages:

In [1]:
! pip install numpy torch torchvision pytorch-ignite

     |████████████████████████████████| 232 kB 40.4 MB/s 


---

In [2]:
import numpy as np
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, datasets, transforms
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
import urllib
import zipfile

In [3]:
# COMMENT THIS OUT IF YOU USE YOUR OWN DATA.
# Download example data into ./data/image-data (4 image files, 2 for "dog", 2 for "cat").
url = "https://github.com/jrieke/traingenerator/raw/main/data/fake-image-data.zip"
zip_path, _ = urllib.request.urlretrieve(url)
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall("data")

# Setup

In [10]:
# INSERT YOUR DATA HERE
# Expected format: One folder per class, e.g.
# train
# --- dogs
# |   +-- lassie.jpg
# |   +-- komissar-rex.png
# --- cats
# |   +-- garfield.png
# |   +-- smelly-cat.png
#
# Example: https://github.com/jrieke/traingenerator/tree/main/data/image-data
train_data = "data/image-data"  # required
val_data = "data/image-data"    # optional
test_data = None                # optional

In [4]:
# Set up hyperparameters.
lr = 0.001
batch_size = 16
num_epochs = 10

In [5]:
# Set up logging.
print_every = 10  # batches

In [6]:
# Set up device.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Preprocessing

In [28]:
import torchvision.transforms.functional as F

class SquarePad:
	def __call__(self, image):
		w, h = image.size
		max_wh = np.max([w, h])
		hp = int((max_wh - w) / 2)
		vp = int((max_wh - h) / 2)
		padding = (hp, vp, hp, vp)
		return F.pad(image, padding, 114, 'constant')

  
def preprocess(data, name):
    if data is None:  # val/test can be empty
        return None

    # Read image files to pytorch dataset.
    transform = transforms.Compose([
        SquarePad(),
        transforms.Grayscale(),
        transforms.Resize(128), 
        transforms.CenterCrop(128), 
        transforms.ToTensor(), 
        
    ])
    dataset = datasets.ImageFolder(data, transform=transform)

    # Wrap in data loader.
    if use_cuda:
        kwargs = {"pin_memory": True, "num_workers": 1}
    else:
        kwargs = {}
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=(name=="train"), **kwargs)
    return loader

In [29]:
train_loader = preprocess(train_data, "train")
val_loader = preprocess(val_data, "val")
test_loader = preprocess(test_data, "test")

# Model

In [38]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.sequential = nn.Sequential(
            nn.Linear(16384,100),# 输入为 128x128 的 灰度图
            nn.ReLU(),
            nn.Linear(100,2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = x.view(-1,16384)
        return self.sequential(x)



In [39]:
# Set up model, loss, optimizer.
model = NeuralNetwork()
model = model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
print(model)

NeuralNetwork(
  (sequential): Sequential(
    (0): Linear(in_features=16384, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)


# Training

In [40]:
# Set up pytorch-ignite trainer and evaluator.
trainer = create_supervised_trainer(
    model,
    optimizer,
    loss_func,
    device=device,
)
metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(loss_func),
}
evaluator = create_supervised_evaluator(
    model, metrics=metrics, device=device
)

In [41]:
@trainer.on(Events.ITERATION_COMPLETED(every=print_every))
def log_batch(trainer):
    batch = (trainer.state.iteration - 1) % trainer.state.epoch_length + 1
    print(
        f"Epoch {trainer.state.epoch} / {num_epochs}, "
        f"batch {batch} / {trainer.state.epoch_length}: "
        f"loss: {trainer.state.output:.3f}"
    )

In [42]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch(trainer):
    print(f"Epoch {trainer.state.epoch} / {num_epochs} average results: ")

    def log_results(name, metrics, epoch):
        print(
            f"{name + ':':6} loss: {metrics['loss']:.3f}, "
            f"accuracy: {metrics['accuracy']:.3f}"
        )

    # Train data.
    evaluator.run(train_loader)
    log_results("train", evaluator.state.metrics, trainer.state.epoch)
    
    # Val data.
    if val_loader:
        evaluator.run(val_loader)
        log_results("val", evaluator.state.metrics, trainer.state.epoch)

    # Test data.
    if test_loader:
        evaluator.run(test_loader)
        log_results("test", evaluator.state.metrics, trainer.state.epoch)

    print()
    print("-" * 80)
    print()

In [ ]:
# Start training.
trainer.run(train_loader, max_epochs=num_epochs)

In [45]:
model.eval()
image = torch.randn(1,1,128,128).to(device)
torch.onnx.export(model,image,"model.onnx",input_names=["data"],output_names=["output"])